In [ ]:
# loading damicm module
import pysimdamicm as ccd
from pysimdamicm import utils, io

# loading other packages
import numpy as np
from matplotlib import pyplot as plt
import ROOT
from astropy.io import fits

In [2]:
file_path = "raw_img_data/avg_Image_5_Low_Temp_109_20250807_170213_15.fz"

In [3]:
hdul = fits.open(file_path)

# show a summary of all extensions
hdul.info()

Filename: raw_img_data/avg_Image_5_Low_Temp_109_20250807_170213_15.fz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     133   ()      
  1  CCD_A         1 CompImageHDU     11   (640, 320)   float64   
  2  CCD_B         1 CompImageHDU     11   (640, 320)   float64   
  3  CCD_D         1 CompImageHDU     11   (640, 320)   float64   
  4  CCD_C         1 CompImageHDU     11   (640, 320)   float64   


In [4]:
# fresh load of the packaged JSON
cfg_file = f"{ccd.__path__[0]}/json/img_5.json"
cfg = ccd.utils.config.Config(cfg_file, simulations=False)

In [5]:
print(ccd.utils.config.json.dumps(cfg.configuration['input'], indent=4, sort_keys=True))

{
    "convention": {
        "Ncols": "NCOL",
        "Npbin": "NPBIN",
        "Nrows": "NROW",
        "Nsbin": "NSBIN",
        "Nskips": "NDCM",
        "ampdir": "VCKDIRN",
        "ampl": "AMPL",
        "exposure_time": "EXPOSURE",
        "itgtime": "TINTEGR",
        "read_time": "MREAD"
    },
    "datetime": {
        "exposure_start": "DATEINI",
        "readout_end": "DATEEND",
        "readout_start": "DATEINI"
    },
    "image": {
        "ACM_multi_extension": true,
        "axis_to_compress": 1,
        "ccd_cols": 640,
        "ccd_rows": 320,
        "correct_leach_bug": false,
        "correct_polarity": false,
        "ext_header": 0,
        "extensions": [
            1,
            2,
            3,
            4
        ],
        "id_col_end": -1,
        "id_col_start": 0,
        "id_row_end": -1,
        "id_row_start": 2,
        "id_skip_end": -1,
        "id_skip_start": 0,
        "n_amp": 1,
        "n_cols_prescan": 0,
        "n_rows_overscan": 0,


In [7]:
# now run
rdata = io.rawdata.BuilderRawData(file_path, cfg.configuration["input"])

WARNING ERROR: Parameter ampdir will be ignored, not found in the data image header (extension 1)
WARNING ERROR: Parameter itgtime will be ignored, not found in the data image header (extension 1)
WARNING ERROR: Parameter ampl will be ignored, not found in the data image header (extension 1)
WARNING ERROR: Parameter read_time will be ignored, not found in the data image header (extension 1)
<panaSKImg> info: Fits file does not contain an skip image, Nskips sets to 1 (from header 500).


In [8]:
rdata.prepare_data()

RawData INFO.
 *********************************************************************** 
 * Define mask for sensor and over/pre-scan regions 'avg_Image_5_Low_Temp_109_20250807_170213_15.fz'
 *********************************************************************** 
  INFO: Rows and column region for the overscan, prescan and Active regions:
     amplifier EXT1 - image: 	 320 rows [0,319] and 640 cols [0,639]
     amplifier EXT1 - mask_full_extension: 	 320 rows [0,319] and 640 cols [0,639]
     amplifier EXT1 - mask_image_active_region: 	 320 rows [0,319] and 640 cols [0,639]

     amplifier EXT2 - image: 	 320 rows [320,639] and 640 cols [0,639]
     amplifier EXT2 - mask_full_extension: 	 320 rows [320,639] and 640 cols [0,639]
     amplifier EXT2 - mask_image_active_region: 	 320 rows [320,639] and 640 cols [0,639]

     amplifier EXT3 - image: 	 320 rows [640,959] and 640 cols [0,639]
     amplifier EXT3 - mask_full_extension: 	 320 rows [640,959] and 640 cols [0,639]
     amplifier E

In [9]:
rdata.image.shape

(1280, 640)

In [13]:
comp = ccd.processes.skipper_analysis.CompressSkipperProcess()
comp.func_to_compress = ['mean']
comp.id_skip_start = 2
comp.execute_process(rdata)

Process <CompressSkipsProcess> INFO. Compressing raw image data using the following statistics: ['mean']
  WARNING. Number of skips = 1, nothing to do!


In [14]:
ped = ccd.processes.skipper_analysis.PedestalSubtractionProcess()

In [17]:
# full image
rdata.execute_process_in_amp = None
ped.in_overscan = True
ped.axis = "row"
ped.n_sigma_to_mask = 10
ped.use_mad = True
# set method
ped.method = 'gauss_fit'
ped.execute_process(rdata)

Process <PedestalSubtractionProcess> INFO. Equalize the image by substracting the pedestal [amplifier None].
   - using the sensitive region


TypeError: the resolved dtypes are not compatible with add.reduce. Resolved (dtype('<U21'), dtype('<U21'), dtype('<U42'))